In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

In [2]:
# Parameters

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

profit = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

time_req = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}
# for dataframe, nested dictionary's inner key becomes index label while the outer key becomes column label

# number of machines down
down = {("Jan","grinder"): 1, ("Feb", "horiDrill"): 2, ("Mar", "borer"): 1,
        ("Apr", "vertDrill"): 1, ("May", "grinder"): 1, ("May", "vertDrill"): 1,
        ("Jun", "planer"): 1, ("Jun", "horiDrill"): 1}

# number of each machine available
installed = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1}

# market limitation of sells
max_sales = {
    ("Jan", "Prod1") : 500,
    ("Jan", "Prod2") : 1000,
    ("Jan", "Prod3") : 300,
    ("Jan", "Prod4") : 300,
    ("Jan", "Prod5") : 800,
    ("Jan", "Prod6") : 200,
    ("Jan", "Prod7") : 100,
    ("Feb", "Prod1") : 600,
    ("Feb", "Prod2") : 500,
    ("Feb", "Prod3") : 200,
    ("Feb", "Prod4") : 0,
    ("Feb", "Prod5") : 400,
    ("Feb", "Prod6") : 300,
    ("Feb", "Prod7") : 150,
    ("Mar", "Prod1") : 300,
    ("Mar", "Prod2") : 600,
    ("Mar", "Prod3") : 0,
    ("Mar", "Prod4") : 0,
    ("Mar", "Prod5") : 500,
    ("Mar", "Prod6") : 400,
    ("Mar", "Prod7") : 100,
    ("Apr", "Prod1") : 200,
    ("Apr", "Prod2") : 300,
    ("Apr", "Prod3") : 400,
    ("Apr", "Prod4") : 500,
    ("Apr", "Prod5") : 200,
    ("Apr", "Prod6") : 0,
    ("Apr", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("Jun", "Prod1") : 500,
    ("Jun", "Prod2") : 500,
    ("Jun", "Prod3") : 100,
    ("Jun", "Prod4") : 300,
    ("Jun", "Prod5") : 1100,
    ("Jun", "Prod6") : 500,
    ("Jun", "Prod7") : 60,
}

holding_cost = 0.5
max_inventory = 100
store_target = 50
hours_per_month = 2*8*24

In [3]:
factory = gp.Model('Factory Planning I')
# The name specified within the parentheses is an optional parameter and is used to give the model a descriptive name.

make = factory.addVars(months, products, lb=0.00, name='Make')
store = factory.addVars(months, products, ub=max_inventory, name='Store')
sell = factory.addVars(months, products, ub=max_sales, name='Sell')
print(sell)

Restricted license - for non-production use only - expires 2025-11-24
{('Jan', 'Prod1'): <gurobi.Var *Awaiting Model Update*>, ('Jan', 'Prod2'): <gurobi.Var *Awaiting Model Update*>, ('Jan', 'Prod3'): <gurobi.Var *Awaiting Model Update*>, ('Jan', 'Prod4'): <gurobi.Var *Awaiting Model Update*>, ('Jan', 'Prod5'): <gurobi.Var *Awaiting Model Update*>, ('Jan', 'Prod6'): <gurobi.Var *Awaiting Model Update*>, ('Jan', 'Prod7'): <gurobi.Var *Awaiting Model Update*>, ('Feb', 'Prod1'): <gurobi.Var *Awaiting Model Update*>, ('Feb', 'Prod2'): <gurobi.Var *Awaiting Model Update*>, ('Feb', 'Prod3'): <gurobi.Var *Awaiting Model Update*>, ('Feb', 'Prod4'): <gurobi.Var *Awaiting Model Update*>, ('Feb', 'Prod5'): <gurobi.Var *Awaiting Model Update*>, ('Feb', 'Prod6'): <gurobi.Var *Awaiting Model Update*>, ('Feb', 'Prod7'): <gurobi.Var *Awaiting Model Update*>, ('Mar', 'Prod1'): <gurobi.Var *Awaiting Model Update*>, ('Mar', 'Prod2'): <gurobi.Var *Awaiting Model Update*>, ('Mar', 'Prod3'): <gurobi.Var *Aw

### Objective function

In [50]:
objective = gp.quicksum(profit[product]*sell[month, product]-holding_cost*store[month, product]
                       for product in products for month in months)
factory.setObjective(objective, GRB.MAXIMIZE)

### Adding constraints
why breaking down balance into 2? becasue different conservation law formula

1st year: make = sell + store

years after 1st year: previous inventory + make = sell + store

In [51]:
for product in products:
    print(make[months[0], product])
    
print(months[months.index('Jan')])
print(months[0])

<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
Jan
Jan


In [52]:
# conservation law
Balance0 = factory.addConstrs((make[months[0], product]==sell[months[0], product]
                               +store[months[0], product] for product in products), name='Initial_Balance')
# for simple linear constraint, parentheses are not required. But, in multiple constraints, explicit parentheses may be necessary. because the variable {(tuple):value}

Balance = factory.addConstrs((store[months[months.index(month)-1], product]
                                    +make[months[months.index(month)], product] ==
                                    sell[months[months.index(month)], product]+store[months[months.index(month)], product] 
                                    for product in products for month in months
                                    if month != months[0]) # exclude 1st year
                                    , name='Balance')
# The index method is used to find the index of elements in a list. 

In [53]:
# inventory target
target_inventory = factory.addConstrs((store[months[-1], product] == store_target for product in products), name='End_Balance')

In [54]:
# machine capacity
machine_capacity = factory.addConstrs((gp.quicksum(time_req[machine][product]*make[month, product]
                                      for product in time_req[machine]) # innermost loop
                                      <= hours_per_month*(installed[machine]-down.get((month, machine),0))
                                      for machine in machines for month in months), # middle loop and outermost loop
                                      name='Capacity')
# single [] for retrieving dictionary of tuples while [][] for nested dictionary
# the above nested loop logic
# for grinder in Jan, sum all the time taken by all products
# for grinder in Feb, sum all the time taken by all products
#..
# for grinder in Jun, sum all the time taken by all products
# for vertDrill in Jan, sum all the time taken by all products
#..
# for vertDrill in Jun, sum all the time taken by all products
#...

In [55]:
factory.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 79 rows, 126 columns and 288 nonzeros
Model fingerprint: 0x2625f534
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [5e+01, 2e+03]
Presolve removed 74 rows and 110 columns
Presolve time: 0.01s
Presolved: 5 rows, 16 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2466500e+05   3.640000e+02   0.000000e+00      0s
Extra simplex iterations after uncrush: 2
       4    9.3715179e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.371517857e+04


In [56]:
factory.printAttr('X')


    Variable            X 
-------------------------
Make[Jan,Prod1]          500 
Make[Jan,Prod2]      888.571 
Make[Jan,Prod3]        382.5 
Make[Jan,Prod4]          300 
Make[Jan,Prod5]          800 
Make[Jan,Prod6]          200 
Make[Feb,Prod1]          700 
Make[Feb,Prod2]          600 
Make[Feb,Prod3]        117.5 
Make[Feb,Prod5]          500 
Make[Feb,Prod6]          300 
Make[Feb,Prod7]          250 
Make[Mar,Prod6]          400 
Make[Apr,Prod1]          200 
Make[Apr,Prod2]          300 
Make[Apr,Prod3]          400 
Make[Apr,Prod4]          500 
Make[Apr,Prod5]          200 
Make[Apr,Prod7]          100 
Make[May,Prod2]          100 
Make[May,Prod3]          600 
Make[May,Prod4]          100 
Make[May,Prod5]         1100 
Make[May,Prod6]          300 
Make[May,Prod7]          100 
Make[Jun,Prod1]          550 
Make[Jun,Prod2]          550 
Make[Jun,Prod4]          350 
Make[Jun,Prod6]          550 
Store[Jan,Prod3]         82.5 
Store[Feb,Prod1]          100 
Store[Feb,Prod

### Production Plan

In [59]:
rows_month = months.copy()
columns_product = products.copy()
make_plan = pd.DataFrame(columns=columns_product, index=rows_month, data=0)
# initiate a dataframe with 0

for month, product in make.keys():
    make_plan.loc[month, product] = np.round(make[month, product].x,1)
make_plan
# .x is a Gurobi attribute used to access the optimal value 

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500,888.6,382.5,300,800,200,0
Feb,700,600.0,117.5,0,500,300,250
Mar,0,0.0,0.0,0,0,400,0
Apr,200,300.0,400.0,500,200,0,100
May,0,100.0,600.0,100,1100,300,100
Jun,550,550.0,0.0,350,0,550,0


### Sales Plan

In [60]:
rows_month = months.copy()
columns_product = products.copy()
sales_plan = pd.DataFrame(columns=columns_product, index=rows_month, data=0)

for month, product in sell.keys():
    sales_plan.loc[month, product] = np.round(sell[month, product].x,1)
sales_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500,888.6,300,300,800,200,0
Feb,600,500.0,200,0,400,300,150
Mar,100,100.0,0,0,100,400,100
Apr,200,300.0,400,500,200,0,100
May,0,100.0,500,100,1000,300,0
Jun,500,500.0,50,300,50,500,50


### Inventory Plan

In [62]:
rows_month = months.copy()
columns_product = products.copy()
inventory_plan = pd.DataFrame(columns=columns_product, index=rows_month, data=0)

for month, product in store.keys():
    inventory_plan.loc[month, product] = np.round(store[month, product].x,1)
inventory_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,0,0,82.5,0,0,0,0
Feb,100,100,0.0,0,100,0,100
Mar,0,0,0.0,0,0,0,0
Apr,0,0,0.0,0,0,0,0
May,0,0,100.0,0,100,0,100
Jun,50,50,50.0,50,50,50,50
